In [1]:
import os
import cv2
import numpy as np
import re
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from scipy.stats import entropy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
#Create an empty dictionary to store the images
bills = {'MX020':[],'MX050':[],'MX100':[],'MX200':[],'MX500':[]}

# define the target size for the images
target_size = (600, 500)

# load the image data into the dictionary and resize each image
for filename in os.listdir('billetes'):
    for bill in bills:
        if bill in filename:
            img = cv2.imread(os.path.join('billetes',filename))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_resized = cv2.resize(img, target_size)
                bills[bill].append(img_resized)



In [ ]:
X = [] # feature vectors
y = [] # labels
for bill, imgs in bills.items():
    for img in imgs:
        X.append(img.flatten())
        y.append(bill)
X = np.array(X)
y = np.array(y)

model = SVC()
model.fit(X, y)



In [ ]:

cap = cv2.VideoCapture(0) # 0 for the default webcam
while True:
    # preprocess each frame to extract the banknote region
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # apply adaptive thresholding to create a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # find the contour with the largest area (i.e., the banknote)
    max_contour = max(contours, key=cv2.contourArea)

    # get the bounding box of the banknote contour
    x, y, w, h = cv2.boundingRect(max_contour)

    # extract the banknote region from the frame
    banknote = frame[y:y+h, x:x+w]

    # apply thresholding, edge detection, or contour detection to extract the banknote region
    # resize the extracted region to the same size as the training images
    resized = cv2.resize(banknote, target_size)

    # use the trained SVC model to predict the denomination of the banknote
    prediction = model.predict(resized.flatten().reshape(1,-1))[0]

    # draw the predicted denomination on the frame
    cv2.putText(frame, prediction, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    # display the frame on the screen
    cv2.imshow('frame', frame)

    # wait for the user to press the 'q' key to exit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()